In [6]:
import pandas as pd

fichier = "online_retail_II.xlsx"

df_0910 = pd.read_excel(fichier, sheet_name="Year 2009-2010")
df_1011 = pd.read_excel(fichier, sheet_name="Year 2010-2011")

df = pd.concat([df_0910, df_1011], ignore_index=True)

df["InvoiceDate"] = pd.to_datetime(df["InvoiceDate"])

df.head()


,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


Infos dataset : 

- Source : UCI Machine Learning Repository  
- Activité : transactions e-commerce d’un détaillant britannique  
- Période couverte : 01/12/2009 → 09/12/2011  
- Granularité : 1 ligne = 1 produit dans 1 facture  
- Nombre de colonnes : 8  
- Colonnes principales : Invoice, StockCode, Description, Quantity, InvoiceDate, Price, Customer ID, Country  


In [12]:

n_lignes, n_colonnes = df.shape
n_invoices = df["Invoice"].nunique()
n_customers = df["Customer ID"].nunique()
n_countries = df["Country"].nunique()

date_min = df["InvoiceDate"].min()
date_max = df["InvoiceDate"].max()

print("===== FICHE SYNTHETIQUE =====")
print(f"Nombre de lignes : {n_lignes}")
print(f"Nombre de colonnes : {n_colonnes}")
print(f"Nombre de factures uniques : {n_invoices}")
print(f"Nombre de clients uniques : {n_customers}")
print(f"Nombre de pays : {n_countries}")
print(f"Période couverte : du {date_min} au {date_max}")


===== FICHE SYNTHETIQUE =====
Nombre de lignes : 1067371
Nombre de colonnes : 8
Nombre de factures uniques : 53628
Nombre de clients uniques : 5942
Nombre de pays : 43
Période couverte : du 2009-12-01 07:45:00 au 2011-12-09 12:50:00


# 2. Dictionnaire des variables

| Nom          | Type        | Description |
|--------------|-------------|-------------|
| Invoice      | string      | Identifiant de facture. Les factures commençant par “C” sont des annulations. |
| StockCode    | string      | Code produit (SKU). |
| Description  | string      | Description texte de l’article. |
| Quantity     | int         | Quantité vendue (peut être négative en cas de retour). |
| InvoiceDate  | datetime    | Date et heure de la transaction. |
| Price        | float       | Prix unitaire hors taxe en GBP. |
| Customer ID  | float/int   | Identifiant client (souvent manquant). |
| Country      | string      | Pays du client. |


In [13]:
dict_vars = pd.DataFrame([
    ["Invoice", "string", "Identifiant de facture (C = annulation)."],
    ["StockCode", "string", "Code article (SKU)."],
    ["Description", "string", "Texte décrivant le produit."],
    ["Quantity", "int", "Quantité (négative = retour)."],
    ["InvoiceDate", "datetime", "Date et heure de la facture."],
    ["Price", "float", "Prix unitaire hors taxe en GBP."],
    ["Customer ID", "float/int", "Identifiant client (souvent manquant)."],
    ["Country", "string", "Pays du client."]
], columns=["Nom", "Type", "Description"])

dict_vars


,Nom,Type,Description
0,Invoice,string,Identifiant de facture (C = annulation).
1,StockCode,string,Code article (SKU).
2,Description,string,Texte décrivant le produit.
3,Quantity,int,Quantité (négative = retour).
4,InvoiceDate,datetime,Date et heure de la facture.
5,Price,float,Prix unitaire hors taxe en GBP.
6,Customer ID,float/int,Identifiant client (souvent manquant).
7,Country,string,Pays du client.


In [14]:
print("===== VALEURS MANQUANTES =====")
na_counts = df.isna().sum()
na_percent = (df.isna().mean() * 100).round(2)

pd.DataFrame({
    "Nb manquants": na_counts,
    "% manquants": na_percent
})


===== VALEURS MANQUANTES =====


,Nb manquants,% manquants
Invoice,0,0.00
StockCode,0,0.00
Description,4382,0.41
Quantity,0,0.00
InvoiceDate,0,0.00
Price,0,0.00
Customer ID,243007,22.77
Country,0,0.00


In [15]:
print("===== DOUBLONS =====")
dup_exact = df.duplicated().sum()
dup_subset = df.duplicated(subset=["Invoice", "StockCode", "Quantity", "Price", "Customer ID", "InvoiceDate"]).sum()

print(f"Doublons exacts : {dup_exact}")
print(f"Doublons sur colonnes clés : {dup_subset}")


===== DOUBLONS =====
Doublons exacts : 34335
Doublons sur colonnes clés : 34337


In [16]:
print("===== ANNULATIONS / RETOURS =====")

df["is_cancelled"] = df["Invoice"].astype(str).str.startswith("C")
df["is_negative_qty"] = df["Quantity"] < 0

print(f"Lignes annulées (factures C) : {df['is_cancelled'].sum()}")
print(f"Pourcentage annulées : {df['is_cancelled'].mean()*100:.2f}%")

print(f"Lignes avec quantité négative : {df['is_negative_qty'].sum()}")
print(f"Pourcentage : {df['is_negative_qty'].mean()*100:.2f}%")


===== ANNULATIONS / RETOURS =====
Lignes annulées (factures C) : 19494
Pourcentage annulées : 1.83%
Lignes avec quantité négative : 22950
Pourcentage : 2.15%


In [17]:
print("===== OUTLIERS =====")

print("\nStatistiques Price :")
print(df["Price"].describe())

print("\nStatistiques Quantity :")
print(df["Quantity"].describe())

print(f"\nNb de prix <= 0 : {(df['Price'] <= 0).sum()}")
print(f"Nb de quantités anormalement élevées (> 99e pct) : {(df['Quantity'] > df['Quantity'].quantile(0.99)).sum()}")


===== OUTLIERS =====

Statistiques Price :
count    1.067371e+06
mean     4.649388e+00
std      1.235531e+02
min     -5.359436e+04
25%      1.250000e+00
50%      2.100000e+00
75%      4.150000e+00
max      3.897000e+04
Name: Price, dtype: float64

Statistiques Quantity :
count    1.067371e+06
mean     9.938898e+00
std      1.727058e+02
min     -8.099500e+04
25%      1.000000e+00
50%      3.000000e+00
75%      1.000000e+01
max      8.099500e+04
Name: Quantity, dtype: float64

Nb de prix <= 0 : 6207
Nb de quantités anormalement élevées (> 99e pct) : 10561


In [18]:
print("===== GRANULARITE TEMPORELLE =====")

df["Date_Jour"] = df["InvoiceDate"].dt.date

print(f"Nombre de jours couverts : {df['Date_Jour'].nunique()}")
print(f"Du {df['InvoiceDate'].min()} au {df['InvoiceDate'].max()}")


===== GRANULARITE TEMPORELLE =====
Nombre de jours couverts : 604
Du 2009-12-01 07:45:00 au 2011-12-09 12:50:00
